# Basic Example

In [1]:
# !pip install gymnasium
# !pip install ray==2.7.0
# !pip install -U ray[rllib]==2.7.0
# !pip install torch
import csbsingleenv
from ray.air.integrations.wandb import WandbLoggerCallback, setup_wandb
import wandb

In [2]:
wandb.init(entity="rl-class-2024",
           project="RL-Projects")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tristntran (rl-class-2024). Use `wandb login --relogin` to force relogin


In [3]:
setup_wandb(config={"entity": "rl-class-2024",
                    "project": "RL-Projects"})

/home/healthcare/.local/lib/python3.10/site-packages/ray/train/_internal/session.py:829: UserWarning: `get_trial_id` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `None`.
  warnings.warn(
/home/healthcare/.local/lib/python3.10/site-packages/ray/train/_internal/session.py:829: UserWarning: `get_trial_name` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `None`.
  warnings.warn(
/home/healthcare/.local/lib/python3.10/site-packages/ray/train/_internal/session.py:829: UserWarning: `get_experiment_name` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `None`.
  warnings.warn(
2024-05-12 17:00:11,046	INFO wandb.py:308 -- Already logged into W&B.


In [4]:
env = csbsingleenv.CodersStrikeBackSingle()

In [5]:
env.reset()
done = False
rewards = []
while not done:
    obs, reward, done, _, _ = env.step(env.action_space.sample())
    rewards.append(reward)
print(sum(rewards))

-10100


In [6]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

2024-05-12 16:52:44,079	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [7]:
if ray.is_initialized():
  ray.shutdown()
ray.init(num_cpus=4)

2024-05-12 16:52:48,769	INFO worker.py:1642 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.7.0


In [8]:
config = (PPOConfig()
          .environment(csbsingleenv.CodersStrikeBackSingle)
          .framework('torch')
          .training(gamma=0.9)
          .rollouts(num_rollout_workers=3)
)

stop = {"timesteps_total": 12000}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop,
                             callbacks=[WandbLoggerCallback(project="RL-Projects",upload_checkpoints=True,
                                                            save_checkpoints=True, group="rl-class-2024")]),
)

tuner.fit()

2024-05-12 16:10:43,131	INFO wandb.py:308 -- Already logged into W&B.
2024-05-12 16:10:43,139	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=1664287) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=1664287) 2024-05-12 16:10:46,166	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a defa

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': 7.307951598279257}, 'default_policy': {'total_loss': 7.307951598279257, 'policy_loss': -0.05196320661790431, 'vf_loss': 7.3480512986559345, 'vf_loss_unclipped': 5121432.709221749, 'vf_explained_var': -6.563005162708795e-05, 'entropy': 4.039082076249601, 'mean_kl_loss': 0.02636333086619353, 'default_optimizer_lr': 5.000000000000001e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.675000011920929}}, 'num_env_steps_sampled': 12000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 12000, 'num_agent_steps_trained': 0}, 'sampler_results': {'episode_reward_max': -10100.0, 'episode_reward_min': -10140.0, 'episode_reward_mean': -10102.31, 'episode_len_mean': 102.31, 'episode_media': {}, 'episodes_this_iter': 39, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mea

# Empty Customized Environment

In [9]:
import csbsingleenv

import sys
# Assuming your Python file is in 'src' under 'my_project' directory
sys.path.append('../..')
import env_customizer

In [10]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [11]:
if ray.is_initialized():
  ray.shutdown()

runtime_env = {"py_modules": ["../.."]}
ray.init(num_cpus=4, runtime_env=runtime_env)

2024-05-12 16:11:22,239	INFO worker.py:1642 -- Started a local Ray instance.
2024-05-12 16:11:22,344	INFO packaging.py:518 -- Creating a file package for local directory '../..'.
2024-05-12 16:11:22,445	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_7acded97c7d60515.zip' (13.71MiB) to Ray cluster...
2024-05-12 16:11:22,510	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_7acded97c7d60515.zip'.


Python version:,3.10.12
Ray version:,2.7.0


In [12]:
config = (PPOConfig()
          .environment(env_customizer.CustomizedEnvironment,
                      env_config={"env" : csbsingleenv.CodersStrikeBackSingle()})
          .framework('torch')
          .training(gamma=0.9)
          .rollouts(num_rollout_workers=3)
)

stop = {"timesteps_total": 12000}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop,
                             callbacks=[WandbLoggerCallback(project="RL-Projects",upload_checkpoints=True,
                                                            save_checkpoints=True, group="rl-class-2024")]),
)

tuner.fit()

2024-05-12 16:11:23,130	INFO wandb.py:308 -- Already logged into W&B.
2024-05-12 16:11:23,136	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=1665329) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=1665329) 2024-05-12 16:11:27,215	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a defa

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': 7.019537636712353}, 'default_policy': {'total_loss': 7.019537636712353, 'policy_loss': -0.05339170935482327, 'vf_loss': 7.0607852718469175, 'vf_loss_unclipped': 4859401.15315894, 'vf_explained_var': -6.320005032553602e-05, 'entropy': 4.318720665567719, 'mean_kl_loss': 0.02698683410676058, 'default_optimizer_lr': 5.000000000000001e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.675000011920929}}, 'num_env_steps_sampled': 12000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 12000, 'num_agent_steps_trained': 0}, 'sampler_results': {'episode_reward_max': -10100.0, 'episode_reward_min': -10200.0, 'episode_reward_mean': -10103.58, 'episode_len_mean': 103.58, 'episode_media': {}, 'episodes_this_iter': 37, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean

# Real Customized Environment

In [6]:
import csbsingleenv

import sys
# Assuming your Python file is in 'src' under 'my_project' directory
sys.path.append('../..')
import env_customizer

2024-05-12 17:00:37,005	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [7]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces, logger

# Note, this is only meant as an example of how to write a custom environment.
# I doubt it'll actually learn well.
class CustomizedRacing:
    def __init__(self):
        min_pos = -200000.0
        max_pos = 200000.0
        min_vel = -2000.0
        max_vel = 2000.0
        screen_max = [16000,9000]
        # This observation space will only have the next checkpoint rather than the next 5.
        self.observation_space = spaces.Box(
            low=np.array([0, -np.pi, min_pos, min_pos, min_vel, min_vel]+[0,0]),
            high=np.array([3, np.pi, max_pos, max_pos, max_vel, max_vel]+screen_max),
            dtype=np.float64
        )

        # This action will be how much to turn, rather than a point to turn towards.
        self.action_space = spaces.Box(
            low = np.array([-1.0, 0.0]),
            high = np.array([1.0, 100]),
            dtype=np.float64
        )

    # We can define and reset any extra variables we want to keep track of here.
    def reset(self):
        self.last_checkpoint = None
        self.last_sqr_dist = None

    def reward(self, env, rew):
        # Add reward for getting closer to the next checkpoint
        cur_dist = csbsingleenv.get_sqr_distance(env.pos, env.next_checkpoint())
        if self.last_sqr_dist is not None and cur_dist < self.last_sqr_dist:
            rew += 500
        self.last_sqr_dist = cur_dist
        
        # Add reward for passing a checkpoint
        cur_checkpoint = env.checkpoint_index
        if self.last_checkpoint is not None and cur_checkpoint != self.last_checkpoint:
            rew += 600
        self.last_checkpoint = cur_checkpoint
        
        if env.failed:
            return -10000
        return rew
    
    # Need to define the traformation.  Should be easy here since we just need to drop the excess data
    def transform_observation(self, env, obs):
        return obs[:8]

    # Need to also transform the angle change into a point to aim at.
    # Will define it as 1000 distance from the angle relative to current pointed direction
    def transform_action(self, env, action):
        preferred_dir = env.theta + action[0]*env.maxSteeringAngle
        dir_vector = 1000*csbsingleenv.angle_to_vector(preferred_dir)
        point_to_aim_at = env.pos + dir_vector
        trans_action = np.array([point_to_aim_at.x, point_to_aim_at.y, action[1]])
        return trans_action
    

In [8]:
env = env_customizer.CustomizedEnvironment(
    env_config={"env":csbsingleenv.CodersStrikeBackSingle(),
                "customizer":CustomizedRacing()})

In [16]:
env.reset()

(array([ 3.00000000e+00, -1.49465519e-01,  6.91600000e+03,  1.52500000e+03,
         0.00000000e+00,  0.00000000e+00,  1.38820000e+04,  4.76000000e+02]),
 {})

In [17]:
env.step(np.array([0,100]))

(array([ 3.00000000e+00, -1.49465519e-01,  7.01500000e+03,  1.51000000e+03,
         8.33000000e+01, -1.19000000e+01,  1.38820000e+04,  4.76000000e+02]),
 -1,
 False,
 False,
 {})

In [9]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [10]:
if ray.is_initialized():
  ray.shutdown()

runtime_env = {"py_modules": ["../.."]}
ray.init(runtime_env=runtime_env)

2024-05-12 17:00:52,307	INFO worker.py:1642 -- Started a local Ray instance.
2024-05-12 17:00:52,410	INFO packaging.py:518 -- Creating a file package for local directory '../..'.
2024-05-12 17:00:52,513	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_7acded97c7d60515.zip' (13.71MiB) to Ray cluster...
2024-05-12 17:00:52,574	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_7acded97c7d60515.zip'.


Python version:,3.10.12
Ray version:,2.7.0


In [11]:
checkpoint_config = air.CheckpointConfig(checkpoint_frequency=5)

In [12]:
config = (PPOConfig()
          .environment(env_customizer.CustomizedEnvironment,
                      env_config={"env" : csbsingleenv.CodersStrikeBackSingle(),
                                 "customizer" : CustomizedRacing()
                                 })
          .framework('torch')
          .training(gamma=0.99,
                    model= {
                              "use_lstm": True,
                              "lstm_cell_size": 64,
                              "max_seq_len": 10
                           })
          .rollouts(num_rollout_workers=20, num_envs_per_worker=5, rollout_fragment_length="auto")
          .resources(num_gpus=4, num_learner_workers=4, num_gpus_per_learner_worker=1)
)
config.num_sgd_iter = 6
stop = {"timesteps_total": 1000000}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop,
                             storage_path="~/Desktop/Tristan/rl-projects/results",
                             checkpoint_config=checkpoint_config,
                             callbacks=[WandbLoggerCallback(project="RL-Projects",upload_checkpoints=True,
                                                            save_checkpoints=True, group="rl-class-2024")],)

)

tuner.fit()


2024-05-12 17:00:56,819	INFO wandb.py:308 -- Already logged into W&B.
2024-05-12 17:00:56,828	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=1708716) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=1708716) 2024-05-12 17:01:00,844	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a defa

# Attempt at loading in pre-trained

In [ ]:
def train_ppo_module_with_bc_finetune(
    dataset: ray.data.Dataset, ppo_module_spec: PPOConfig
) -> str:
    """Train an Actor with BC finetuning on dataset.

    Args:
        dataset: The dataset to train on.
        module_spec: The module spec of the PPORLModule that will be trained
            after its encoder and policy networks are pretrained with BC.

    Returns:
        The path to the checkpoint of the pretrained PPORLModule.
    """
    batch_size = 512
    learning_rate = 1e-3
    num_epochs = 10

    module = ppo_module_spec.build()
    # We want to pretrain the encoder and policy networks of the RLModule. We don't want
    # to pretrain the value network. The actor will use the Categorical distribution,
    # as its output distribution since we are training on the CartPole environment which
    # has a discrete action space.
    BCActorNetwork = BCActor(module.encoder, module.pi, torch.distributions.Categorical)
    optim = torch.optim.Adam(BCActorNetwork.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        for batch in dataset.iter_torch_batches(
            batch_size=batch_size, dtypes=torch.float32
        ):
            action_dist = BCActorNetwork(batch)
            loss = -torch.mean(action_dist.log_prob(batch["actions"]))
            optim.zero_grad()
            loss.backward()
            optim.step()
        print(f"Epoch {epoch} loss: {loss.detach().item()}")

    checkpoint_dir = tempfile.mkdtemp()
    module.save_to_checkpoint(checkpoint_dir)
    return checkpoint_dir

In [ ]:
ds = ray.data.read_json("rl-projects/racing/racing_pretrain.json")

config = (PPOConfig()
          .environment(env_customizer.CustomizedEnvironment,
                      env_config={"env" : csbsingleenv.CodersStrikeBackSingle(),
                                 "customizer" : CustomizedRacing()
                                 })
          .framework('torch')
          .training(gamma=0.99,
                    model= {
                              "use_lstm": True,
                              "lstm_cell_size": 64,
                              "max_seq_len": 10
                           })
          .rollouts(num_rollout_workers=20, num_envs_per_worker=5, rollout_fragment_length="auto")
          .resources(num_gpus=4, num_learner_workers=4, num_gpus_per_learner_worker=1)
)


module_checkpoint_path = train_ppo_module_with_bc_finetune(ds, module_spec)
